In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from bitcoin_deep_learning.call_api import ApiCall
import numpy as np
import pandas as pd

In [3]:
df = ApiCall().read_local(data="train")

In [4]:
df

,date,[AVG]_[NH]_mean_hash_rate,[+]_[NH]_Days_Till_Halving,[//]_[AV]_Stock-to-Flow_Ratio,[+]_[NH]_Circulating_Supply,[+]_[NH]_Issuance,[+]_[NH]_Number_of_Active_Addresses,[+]_[NH]_Mean_Block_Interval,[+]_[NH]_Number_of_Transactions,[+]_[NH]_Number_of_Addresses_with_a_Non-Zero_Balance,...,[%]_Bitcoin_growth_rate_on_Horizon=7,fear_greed_value,[+]_[NH]_Number_of_Addresses_with_Balance_0.01 - 0.1,[+]_[NH]_Number_of_Addresses_with_Balance_0.1 - 1,[+]_[NH]_Number_of_Addresses_with_Balance_1 - 10,[+]_[NH]_Number_of_Addresses_with_Balance_10 - 100,[+]_[NH]_Number_of_Addresses_with_Balance_100 - 1k,[+]_[NH]_Number_of_Addresses_with_Balance_1k - 10k,[+]_[T]_Bitcoin_Price,[%]_Bitcoin_growth_rate_on_Horizon=7.1
0,2018-02-01,2.054109e+19,830,5642.903220,16839700.00,2000.00,842398,377,257504,27026398,...,-0.096230,30,4226306,1722331,544769,132314,15779,1529,9014.026207,-0.096230
1,2018-02-02,2.170824e+19,829,5641.695528,16841800.00,2100.00,854253,385,235750,26955617,...,-0.013968,15,4198274,1704275,543489,132213,15815,1521,8799.121911,-0.013968
2,2018-02-03,2.073919e+19,828,5641.015128,16843775.00,1975.00,714655,384,194733,26886977,...,-0.078603,40,4173275,1692828,542447,132021,15812,1523,9208.459919,-0.078603
3,2018-02-04,2.280248e+19,827,5632.191528,16846000.00,2225.00,720904,349,173509,26753036,...,-0.012779,24,4103680,1674480,540698,132009,15830,1532,8176.460616,-0.012779
4,2018-02-05,2.411984e+19,826,5626.186626,16848312.50,2312.50,903239,338,216178,26562089,...,0.224166,11,4036425,1668996,541030,131987,15813,1519,6891.934414,0.224166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401,2021-12-03,1.895333e+20,877,107683.375209,18890443.75,1050.00,1009846,335,289971,39196439,...,-0.137606,31,6033537,2468409,663251,132374,13950,2057,53765.796483,-0.137606
1402,2021-12-04,1.690595e+20,876,107627.370170,18891412.50,968.75,966373,365,253519,39269124,...,0.007926,25,6045160,2473113,664015,132441,13979,2061,48962.920993,0.007926
1403,2021-12-05,1.869669e+20,875,107497.322411,18892450.00,1037.50,966666,342,215047,39150342,...,0.018836,18,6056667,2476280,664358,132558,13969,2067,49174.902908,0.018836
1404,2021-12-06,1.883618e+20,874,107272.444575,18893525.00,1075.00,1059561,358,284295,39078981,...,-0.079764,16,6056432,2478318,664726,132764,13971,2059,50492.435596,-0.079764


In [844]:
y_true = df["[+]_[T]_Bitcoin_Price"].copy().tail(90)

In [845]:
y_pred = y_true

In [846]:
y_true_sideways = df["[+]_[T]_Bitcoin_Price"].copy()[1185:1275:]
y_true_sideways

1196    46803.830522
1197    46406.331544
1198    43382.147859
1199    42985.843096
1200    37151.953936
            ...     
1281    43785.481819
1282    46284.973982
1283    45594.076272
1284    45593.635931
1285    44428.288467
Name: [+]_[T]_Bitcoin_Price, Length: 90, dtype: float64

In [847]:
y_pred_sideways = y_true_sideways

In [848]:
y_true_bear = df["[+]_[T]_Bitcoin_Price"].copy()[1350:1440:]
y_true_bear

1361    58639.680461
1362    60572.271244
1363    62234.031011
1364    61735.370670
1365    61366.999882
            ...     
1446    38137.472697
1447    37936.328777
1448    38455.689663
1449    38784.502299
1450    36924.102394
Name: [+]_[T]_Bitcoin_Price, Length: 90, dtype: float64

In [849]:
y_pred_bear = y_true_bear

In [850]:
y_true_bull = df["[+]_[T]_Bitcoin_Price"].copy()[1260:1350:]
y_true_bull

1271    40037.521622
1272    42080.754911
1273    41692.757323
1274    40135.332001
1275    39183.672449
            ...     
1356    60703.429541
1357    61342.908255
1358    60881.947565
1359    62967.413023
1360    60449.355303
Name: [+]_[T]_Bitcoin_Price, Length: 90, dtype: float64

In [851]:
y_pred_bull = y_true

In [852]:
def compute_roi(play_strategy):
    return (list(play_strategy)[-1] / list(play_strategy)[0]) - 1

def compute_returns(play_strategy):
    return play_strategy.pct_change()[1:]

def compute_sharpe_ratio(play_strategy):
    if compute_returns(play_strategy).sum() == 0:
        return 0
    else:
        return compute_roi(play_strategy) / compute_returns(play_strategy).std()

# Whale

In [853]:
def play_whale_strategy(y_true,
                        y_pred, 
                        total_investment = 3000,
                        investment_horizon = 7,
                        buy_threshold = 0.15,
                        sell_threshold = - 0.05,
                        exchange_fee = 0.005,
                        tax_rate = 0.30):
    """
    Whale strategy:
    (1) Assess daily if the predicted price will reach the buy_threshold or the sell_threshold over the investment_horizon.
    (2) Invest total_investment if price is predicted to increase by at least the buy_threshold over the investment_horizon.
    (3) After reaching the investment horizon, sells total_investment if price is predicted to decrease by at least the sell_threshold over the investment_horizon.
    (4) Repeat process from (1).
    """
    
    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []
    
    # Trackers
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    investment = 0
    reassessment_day = 0
    taxes = 0
    counter = 0
    
    # Loop
    for value in list(y_true):
        
        if len(list(y_pred)) > counter + investment_horizon:
        
            if ((list(y_pred)[counter + investment_horizon] / value) -1) > buy_threshold:
                
                if usd_balance > 0:
                    
                    investment = usd_balance - (usd_balance * exchange_fee)
                    
                    # Tax-prep-start
                    investments.append(investment)
                    cost_basis.append(value)
                    # Tax-prep-end
                    
                    usd_balance = 0
                    btc_usd_balance += investment
                    btc_balance += btc_usd_balance / value
                    reassessment_day = counter + investment_horizon
                    
                    ### print(f"price bought {value}")
                    ### print(f"price predicted {list(y_pred)[counter + investment_horizon]}")
                
                else:
                    btc_usd_balance = btc_balance * value

            if usd_balance == 0:
                
                if counter >= reassessment_day:
                    
                    if ((list(y_pred)[counter + investment_horizon] / value) -1) < sell_threshold:
                        
                        btc_usd_balance = btc_balance * value
                        usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                        btc_usd_balance = 0
                        btc_balance = 0
                        
                        ### print(f"price sold {value}")

                        # Tax-calc-start
                        wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                        taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                        investments.clear()
                        cost_basis.clear()
                        # Tax-calc-end

                    else: 
                        btc_usd_balance = btc_balance * value

                else:
                    btc_usd_balance = btc_balance * value
                
        if len(list(y_pred)) <= counter + investment_horizon:
            
            if counter >= reassessment_day:
                
                if btc_usd_balance > 0:
                    
                    btc_usd_balance = btc_balance * value
                    usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                    btc_usd_balance = 0
                    btc_balance = 0

                    ### print(f"price sold {value}")

                    # Tax-calc-start
                    wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                    taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                    investments.clear()
                    cost_basis.clear()
                    # Tax-calc-end
                
                else:
                    btc_usd_balance = btc_balance * value
            else:
                btc_usd_balance = btc_balance * value

        daily_usd_position.append(usd_balance)
        daily_btc_position.append(btc_balance)
        daily_btc_usd_position.append(btc_usd_balance)
        daily_portfolio_position.append(usd_balance + btc_usd_balance)
                
        counter += 1  

        ### print(counter)
       
    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end
    
    ### print(np.array(taxable_basis).sum() * tax_rate)
                     
    return pd.Series(daily_portfolio_position)
    

# Trader

In [854]:
def play_trader_strategy(y_true,
                        y_pred, 
                        total_investment = 3000,
                        investment_horizon = 7,
                        buy_threshold = 0.10,
                        sell_threshold = -0.05,
                        exchange_fee = 0.005,
                        tax_rate = 0.30):
    """
    Trader strategy:
    (1) Assess daily if the predicted price will reach the buy_threshold or the sell_threshold over the investment_horizon.
    (2) Invest total_investment if price is predicted to increase by at least the buy_threshold over the investment_horizon.
    (3) After reaching the investment horizon, sells total_investment if price is predicted to decrease by at least the sell_threshold over the investment_horizon.
    (4) Repeat process from (1).
    """
    
    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []
    
    # Trackers
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    investment = 0
    reassessment_day = 0
    taxes = 0
    counter = 0
    
    # Loop
    for value in list(y_true):
        
        if len(list(y_pred)) > counter + investment_horizon:
        
            if ((list(y_pred)[counter + investment_horizon] / value) -1) > buy_threshold:
                
                if usd_balance > 0:
                    
                    investment = usd_balance - (usd_balance * exchange_fee)
                    
                    # Tax-prep-start
                    investments.append(investment)
                    cost_basis.append(value)
                    # Tax-prep-end
                    
                    usd_balance = 0
                    btc_usd_balance += investment
                    btc_balance += btc_usd_balance / value
                    reassessment_day = counter + investment_horizon
                    
                    ### print(f"price bought {value}")
                    ### print(f"price predicted {list(y_pred)[counter + investment_horizon]}")
                
                else:
                    btc_usd_balance = btc_balance * value

            if usd_balance == 0:
                
                if counter >= reassessment_day:
                    
                    if ((list(y_pred)[counter + investment_horizon] / value) -1) < sell_threshold:
                        
                        btc_usd_balance = btc_balance * value
                        usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                        btc_usd_balance = 0
                        btc_balance = 0
                        
                        ### print(f"price sold {value}")

                        # Tax-calc-start
                        wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                        taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                        investments.clear()
                        cost_basis.clear()
                        # Tax-calc-end

                    else: 
                        btc_usd_balance = btc_balance * value

                else:
                    btc_usd_balance = btc_balance * value
                
        if len(list(y_pred)) <= counter + investment_horizon:
            
            if counter >= reassessment_day:
                
                if btc_usd_balance > 0:
                    
                    btc_usd_balance = btc_balance * value
                    usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                    btc_usd_balance = 0
                    btc_balance = 0

                    ### print(f"price sold {value}")

                    # Tax-calc-start
                    wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                    taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                    investments.clear()
                    cost_basis.clear()
                    # Tax-calc-end
                
                else:
                    btc_usd_balance = btc_balance * value
            else:
                btc_usd_balance = btc_balance * value
        
        daily_usd_position.append(usd_balance)
        daily_btc_position.append(btc_balance)
        daily_btc_usd_position.append(btc_usd_balance)
        daily_portfolio_position.append(usd_balance + btc_usd_balance)

        counter += 1  

        ### print(counter)
       
    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end
    
    ### print(taxable_basis)
                     
    return pd.Series(daily_portfolio_position)
    

## Trader OLD VERSION - Not working

In [855]:
def play_trader_strategy_2(y_true,
                           y_pred,
                           total_investment = 400,
                           investment_frequency = 7,
                           buy_threshold = 0.05,
                           sell_threshold = -0.05,
                           buy_multiplicator = 5,
                           sell_multiplicator = 5,
                           exchange_fee = 0.005,
                           tax_rate = 0.30):
    """
    Takes y_true and y_pred pd.Series, total_investment, investment_frequency (in days), investment_threshold, exchange_fee.
    Returns a pd.Series of the daily portfolio positions).
    """

    # Enter total USD amount to be invested over the period (i.e., budget)
    # total_investment = 400

    # Enter the investment horizon in days (e.g., 7 for a weekly investement strategy)
    # investment_frequency = 7

    # Enter a buy threshold (i.e., buy if prediction is above/below a certain percentage)
    # buy_threshold = 0.05

    # Enter a sell threshold (i.e., sell if prediction is above/below a certain percentage)
    # sell_threshold = -0.10

    # Enter a buyer multiplicator (e.g., 2 times the price increase rate predicted: 2 * 15%)
    # buy_multiplicator = 2

    # Enter a seller multiplicator (e.g., 2 times the price decrease rate predicted: 2 * 15%)
    # sell_multiplicator = 3

    # Enter the exchange fee applicable to each trade (e.g., 0.5% on Coinbase Pro)
    # exchange_fee = 0.005

    # Enter the applicable tax rate (e.g., 30% for financial gains in France)
    # tax_rate = 0.30

    # Periodic investment amount (e.g., total investment / number of weeks)
    investment_amount = total_investment / (len(y_true) / investment_frequency)

    # List of daily percent change of y_true
    y_true_daily_pct_change = pd.Series(y_true).pct_change()

    # List of daily_portfolio position (usd + btc_usd)
    daily_portfolio_position = []

    # List of the price percent change over the investment horizon (e.g., D7 price / D0 price)
    y_true_percent_change = [(value / list(y_true)[index - investment_frequency])-1 for index, value in enumerate(list(y_true)) if index % investment_frequency == 0][1:]

    # List of returns over the investment horizon (e.g., D7 price / D0 price)
    investment_frequency_returns = []

    # USD balance
    usd_balance = total_investment

    # Bitcoin balance in USD
    btc_usd_balance = 0

    # Cost_basis list for tax calculation purposes
    cost_basis = []

    # Amount bought for cost_basis weighting
    amount_bought = []

    # Taxable basis (i.e., profits taken)
    taxable_basis = []

    taxes = 0

    # Counters for indexes during for-loop
    counter = 0
    counter_percent_change = 0

    # Invest at each period of the investment horizon (e.g., weekly)
    for value in y_pred[::investment_frequency]:

        # Buy/sell only based on a prediction
        if len(y_pred[::investment_frequency]) > counter + 1:
            #print((list(y_pred)[::investment_frequency][counter + 1] / list(y_true)[::investment_frequency][counter]) - 1 < -investment_threshold)

            predicted_move_ratio = (list(y_pred)[::investment_frequency][counter + 1] / list(y_true)[::investment_frequency][counter]) - 1

            #print(predicted_move_ratio)

            if predicted_move_ratio > buy_threshold:
                if usd_balance <= 0:
                    pass
                else:

                    ### Tax ###
                    cost_basis.append(list(y_true)[::investment_frequency][counter])
                    amount_bought.append(usd_balance * (predicted_move_ratio * buy_multiplicator))
                    ### Tax ###

                    btc_usd_balance += usd_balance * (predicted_move_ratio * buy_multiplicator) - ((usd_balance * (predicted_move_ratio * buy_multiplicator)) * exchange_fee)
                    usd_balance -= usd_balance * (predicted_move_ratio * buy_multiplicator)
                    # print(btc_usd_balance)


                # TODO (buy because price is predicted to go up)

            if predicted_move_ratio < sell_threshold:
                if btc_usd_balance == 0:
                    pass

                else:

                    ### Tax ###
                    amount_sold = btc_usd_balance * (- predicted_move_ratio * sell_multiplicator) - ((btc_usd_balance * (- predicted_move_ratio * sell_multiplicator)) * exchange_fee)
                    wa_cost_basis = (np.array(amount_bought) * np.array(cost_basis)).sum() / np.array(amount_bought).sum()
                    taxable_basis.append(amount_sold * ((list(y_true)[::investment_frequency][counter] / wa_cost_basis) - 1))

                    for index in range(len(amount_bought)):
                        amount_bought[index] -= amount_sold / len(amount_bought)

                    ### Tax ###
                    # print(amount_bought)

                    usd_balance += btc_usd_balance * (- predicted_move_ratio * sell_multiplicator) - ((btc_usd_balance * (- predicted_move_ratio * sell_multiplicator)) * exchange_fee)
                    btc_usd_balance -= btc_usd_balance * (- predicted_move_ratio * sell_multiplicator)
                    #print(btc_usd_balance)
                # TODO (buy because price is predicted to go down)


        for days in range(investment_frequency):
            if days == 0:
                daily_portfolio_position.append(usd_balance + btc_usd_balance)
            else:
                daily_portfolio_position.append(usd_balance + (btc_usd_balance + (btc_usd_balance * list(y_true_daily_pct_change)[days])))
            #print(btc_usd_balance)
        counter += 1

        # print(btc_usd_balance)
        # print(investment_frequency_returns)

    # Return On Investment = btc_usd_balance / total invested
    # roi = (btc_usd_balance / total_investment) -1


    # Return On Investment after taxes on trades

    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes

    # roi_after_taxes_on_trades = ((btc_usd_balance + usd_balance - taxes) / total_investment) - 1

    #assert btc_usd_balance == daily_portfolio_position[- investment_frequency]

    #return pd.Series(daily_portfolio_position)
    return pd.Series(daily_portfolio_position)

## HODLER OLD VERSION - NOT WORKING

In [856]:
def play_hodl_strategy(y_true,
                       y_pred,
                       total_investment = 3000,
                       investment_frequency = 7,
                       exchange_fee = 0.005):
    """
    Takes y_true and y_pred pd.Series, total_investment, investment_frequency (in days), exchange_fee.
    Returns a pd.Series of the daily portfolio positions).
    """

    # Periodic investment amount (e.g., total investment / number of weeks)
    investment_amount = total_investment / (len(y_true) / investment_frequency)

    # List of daily percent change of y_true
    y_true_daily_pct_change = pd.Series(y_true).pct_change()

    # List of daily_portfolio position (usd + btc_usd)
    daily_portfolio_position = []

    # List of the price percent change over the investment horizon (e.g., D7 price / D0 price)
    y_true_percent_change = [(value / list(y_true)[index - investment_frequency])-1 for index, value in enumerate(list(y_true)) if index % investment_frequency == 0][1:]

    # List of returns over the investment horizon (e.g., D7 price / D0 price)
    investment_frequency_returns = []

    # USD balance
    usd_balance = total_investment

    # Bitcoin balance in USD
    btc_usd_balance = 0

    # Counters for indexes during for-loop
    counter = 0
    counter_percent_change = 0

    # Invest at each period of the investment horizon (e.g., weekly)
    for value in y_true[::investment_frequency]:

        usd_balance -= investment_amount

        if btc_usd_balance > 0:
            investment_frequency_returns.append(btc_usd_balance * y_true_percent_change[counter_percent_change])
            btc_usd_balance += investment_amount + investment_frequency_returns[counter_percent_change] - (investment_amount * exchange_fee)
            counter_percent_change += 1

        else:
            btc_usd_balance += investment_amount - (investment_amount * exchange_fee)

        for days in range(investment_frequency):
            if days == 0:
                daily_portfolio_position.append(usd_balance + btc_usd_balance)
            else:
                daily_portfolio_position.append(usd_balance + (btc_usd_balance + (btc_usd_balance * list(y_true_daily_pct_change)[days])))


        counter += 1

        print(usd_balance)
        #print(investment_frequency_returns)

    # Return On Investment = btc_usd_balance / total invested
    # roi = (btc_usd_balance / total_investment) -1

    # assert btc_usd_balance == daily_portfolio_position[- investment_frequency]

    return pd.Series(daily_portfolio_position)

# New HODLER

## HODLER, works but not tracking daily positions (only weekly)

In [857]:
def play_hodler_W_strategy(y_true,
                        y_pred, 
                        total_investment = 3000,
                        investment_horizon = 7,
                        exchange_fee = 0.005,
                        tax_rate = 0.30):
    """
    Hodler strategy:
    (1) Buy the same amount following a regular investment horizon without consideration of price prediction.
    (2) Invest equal portions of total_investment at each investment horizon step (i.e., Dollar Cost Averaging - "DCA").
    (3) Never sells ("diamond hands"). However, in order to calculate potential returns, a tax rate (in case of profit taking) will be applied on the last day of the hodler's portfolio. 
    """
    
    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []
    
    # Trackers
    investment = total_investment / (len(list(y_true)) / investment_horizon)
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    taxes = 0
    counter = 0
    
    # Loop
    for value in list(y_true)[::investment_horizon]:
        
        if counter == 0:
            usd_balance -= investment
            btc_usd_balance += investment - (investment * exchange_fee)
            btc_balance = btc_usd_balance / value
            
            # Tax-prep-start
            investments.append(investment)
            cost_basis.append(value)
            # Tax-prep-end
            
        if counter > 0 and counter < len(list(y_true)[::investment_horizon]) - 1 and usd_balance > investment:
            btc_usd_balance = btc_balance * value
            usd_balance -= investment
            btc_usd_balance += investment - (investment * exchange_fee)
            btc_balance = btc_usd_balance / value
            
            # Tax-prep-start
            investments.append(investment)
            cost_basis.append(value)
            # Tax-prep-end
        
        if counter > 0 and counter == len(list(y_true)[::investment_horizon]) - 1:
            btc_usd_balance = btc_balance * value
            usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
            btc_usd_balance = 0
            btc_balance = 0
            
            # Tax-calc-start
            wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
            taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
            investments.clear()
            cost_basis.clear()
            # Tax-calc-end
        
        daily_usd_position.append(usd_balance)
        daily_btc_position.append(btc_balance)
        daily_btc_usd_position.append(btc_usd_balance)
        daily_portfolio_position.append(usd_balance + btc_usd_balance)
        
        counter += 1
    
        ### print(counter)
        
    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end
    
    return pd.Series(daily_portfolio_position)


In [923]:
len(list(y_true))  - ((12 * 7) - ((12 * 7) - len(list(y_true))))
                  
                  

0

In [1027]:
len(list(y_true)[::7]) - 1

12

In [1034]:
def play_hodler_strategy(y_true,
                        y_pred, 
                        total_investment = 3000,
                        investment_horizon = 7,
                        exchange_fee = 0.005,
                        tax_rate = 0.30):
    """
    Hodler strategy:
    (1) Buys the same amount following a regular investment horizon without consideration of price prediction.
    (2) Invests equal portions of total_investment at each investment horizon step (i.e., Dollar Cost Averaging - "DCA").
    (3) Never sells ("diamond hands"). However, in order to calculate potential returns, a tax rate (in case of profit taking) will be applied on the last day of the hodler's portfolio. 
    """
    
    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []
    
    # Trackers
    investment = total_investment / (len(list(y_true)) / investment_horizon)
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    taxes = 0
    counter = 0
    
    # Loop
    for value in list(y_true)[::investment_horizon]:
        
        if counter == 0:
            usd_balance -= investment
            btc_usd_balance += investment - (investment * exchange_fee)
            btc_balance = btc_usd_balance / value

            # Tax-prep-start
            investments.append(investment)
            cost_basis.append(value)
            # Tax-prep-end
            
        if counter > 0 and counter < len(list(y_true)[::investment_horizon]) - 1 and usd_balance > investment:
            btc_usd_balance = btc_balance * value
            usd_balance -= investment
            btc_usd_balance += investment - (investment * exchange_fee)
            btc_balance = btc_usd_balance / value
            
            # Tax-prep-start
            investments.append(investment)
            cost_basis.append(value)
            # Tax-prep-end
        
        if counter > 0 and counter == len(list(y_true)[::investment_horizon]) - 1:
            btc_usd_balance = btc_balance * value
            usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
            btc_usd_balance = 0
            btc_balance = 0
            
            # Tax-calc-start
            wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
            taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
            investments.clear()
            cost_basis.clear()
            # Tax-calc-end
        
        if counter < int(len(list(y_true)) / investment_horizon):
            
            for day in range(investment_horizon):
                daily_usd_position.append(usd_balance)
                daily_btc_position.append(btc_balance)
                btc_usd_balance = btc_balance * list(y_true)[counter * investment_horizon + day]
                daily_btc_usd_position.append(btc_usd_balance)
                daily_portfolio_position.append(usd_balance + btc_usd_balance)
        
        if counter == int(len(list(y_true)) / investment_horizon):
            for day in range((counter) - (len(list(y_true)) - (counter * investment_horizon))):
                daily_usd_position.append(usd_balance)
                daily_btc_position.append(btc_balance)
                btc_usd_balance = btc_balance * list(y_true)[counter * investment_horizon + day]
                daily_btc_usd_position.append(btc_usd_balance)
                daily_portfolio_position.append(usd_balance + btc_usd_balance)

        ### print(counter)
        
        counter += 1
        
    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end
    
    ### print(np.array(taxable_basis).sum() * tax_rate)
    
    return pd.Series(daily_portfolio_position)


In [1035]:
play_hodler_strategy(y_true_bull, y_pred_bull)

390.4144070726492


0     2998.833333
1     3010.681486
2     3008.431594
3     2999.400509
4     2993.882095
         ...     
85    3926.551961
86    3926.551961
87    3926.551961
88    3926.551961
89    3536.137554
Length: 90, dtype: float64

# HODLER WHALE STRATEGY

In [860]:
def play_hodler_whale_strategy(y_true,
                        y_pred, 
                        total_investment = 3000,
                        investment_horizon = 7,
                        buy_threshold = 0.15,
                        sell_threshold = - 1,
                        exchange_fee = 0.005,
                        tax_rate = 0.30):
    """
    Trader strategy:
    (1) Assess daily if the predicted price will reach the buy_threshold or the sell_threshold over the investment_horizon.
    (2) Invest total_investment if price is predicted to increase by at least the buy_threshold over the investment_horizon.
    (3) After reaching the investment horizon, sells total_investment if price is predicted to decrease by at least the sell_threshold over the investment_horizon.
    (4) Repeat process from (1).
    """
    
    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []
    
    # Trackers
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    investment = 0
    reassessment_day = 0
    taxes = 0
    counter = 0
    
    # Loop
    for value in list(y_true):
        
        if len(list(y_pred)) > counter + investment_horizon:
        
            if ((list(y_pred)[counter + investment_horizon] / value) -1) > buy_threshold:
                
                if usd_balance > 0:
                    
                    investment = usd_balance - (usd_balance * exchange_fee)
                    
                    # Tax-prep-start
                    investments.append(investment)
                    cost_basis.append(value)
                    # Tax-prep-end
                    
                    usd_balance = 0
                    btc_usd_balance += investment
                    btc_balance += btc_usd_balance / value
                    reassessment_day = counter + investment_horizon
                    
                    ### print(f"price bought {value}")
                    ### print(f"price predicted {list(y_pred)[counter + investment_horizon]}")
                
                else:
                    btc_usd_balance = btc_balance * value

            if usd_balance == 0:
                
                if counter >= reassessment_day:
                    
                    if ((list(y_pred)[counter + investment_horizon] / value) -1) < sell_threshold:
                        
                        btc_usd_balance = btc_balance * value
                        usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                        btc_usd_balance = 0
                        btc_balance = 0
                        
                        ### print(f"price sold {value}")

                        # Tax-calc-start
                        wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                        taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                        investments.clear()
                        cost_basis.clear()
                        # Tax-calc-end

                    else: 
                        btc_usd_balance = btc_balance * value

                else:
                    btc_usd_balance = btc_balance * value
                
        if len(list(y_pred)) <= counter + investment_horizon:
            
            if counter >= reassessment_day:
                
                if btc_usd_balance > 0:
                    
                    btc_usd_balance = btc_balance * value
                    usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                    btc_usd_balance = 0
                    btc_balance = 0

                    ### print(f"price sold {value}")

                    # Tax-calc-start
                    wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                    taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                    investments.clear()
                    cost_basis.clear()
                    # Tax-calc-end
                
                else:
                    btc_usd_balance = btc_balance * value
            else:
                btc_usd_balance = btc_balance * value
        
        daily_usd_position.append(usd_balance)
        daily_btc_position.append(btc_balance)
        daily_btc_usd_position.append(btc_usd_balance)
        daily_portfolio_position.append(usd_balance + btc_usd_balance)
        
        counter += 1  

        ### print(counter)
       
    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end
    
    ### print(np.array(taxable_basis).sum() * tax_rate)
                     
    return pd.Series(daily_portfolio_position)
    

In [861]:
play_trader_strategy(y_true, y_pred)

0     3000.000000
1     3000.000000
2     3000.000000
3     3000.000000
4     3000.000000
         ...     
85    3366.697931
86    3836.799019
87    3836.799019
88    3836.799019
89    3526.878299
Length: 90, dtype: float64

In [862]:
def play_charles_strategy(y_true,
                          y_pred,
                          total_investment = 3000,
                          investment_horizon = 7,
                          buy_threshold = 0.10,
                          sell_threshold = - 0.10,
                          exchange_fee = 0.005,
                          tax_rate = 0.30):
    """
    Charles strategy:
    (1) Assess daily if the predicted price will reach the buy_threshold or the sell_threshold over the investment_horizon.
    (2) Invest total_investment if price is predicted to increase by at least the buy_threshold over the investment_horizon.
    (3) After reaching the investment horizon, sells total_investment if price is predicted to decrease by at least the sell_threshold over the investment_horizon.
    (4) Repeat process from (1).
    """

    # Lists
    daily_usd_position = []
    daily_btc_position = []
    daily_btc_usd_position = []
    daily_portfolio_position = []
    investments = []
    cost_basis = []
    taxable_basis = []

    # Trackers
    usd_balance = total_investment
    btc_balance = 0
    btc_usd_balance = 0
    investment = 0
    reassessment_day = 0
    taxes = 0
    counter = 0

    # Loop
    for value in list(y_true):

        if len(list(y_pred)) > counter + investment_horizon:

            if ((list(y_pred)[counter + investment_horizon] / value) -1) > buy_threshold:

                if usd_balance > 0:

                    investment = usd_balance - (usd_balance * exchange_fee)

                    # Tax-prep-start
                    investments.append(investment)
                    cost_basis.append(value)
                    # Tax-prep-end

                    usd_balance = 0
                    btc_usd_balance += investment
                    btc_balance += btc_usd_balance / value
                    reassessment_day = counter + investment_horizon

                    ### print(f"price bought {value}")
                    ### print(f"price predicted {list(y_pred)[counter + investment_horizon]}")

                else:
                    btc_usd_balance = btc_balance * value

            if usd_balance == 0:

                if counter >= reassessment_day:

                    if ((list(y_pred)[counter + investment_horizon] / value) -1) < sell_threshold:

                        btc_usd_balance = btc_balance * value
                        usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                        btc_usd_balance = 0
                        btc_balance = 0

                        ### print(f"price sold {value}")

                        # Tax-calc-start
                        wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                        taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                        investments.clear()
                        cost_basis.clear()
                        # Tax-calc-end

                    else:
                        btc_usd_balance = btc_balance * value

                else:
                    btc_usd_balance = btc_balance * value

        if len(list(y_pred)) <= counter + investment_horizon:

            if counter >= reassessment_day:

                if btc_usd_balance > 0:

                    btc_usd_balance = btc_balance * value
                    usd_balance += btc_usd_balance - (btc_usd_balance * exchange_fee)
                    btc_usd_balance = 0
                    btc_balance = 0

                    ### print(f"price sold {value}")

                    # Tax-calc-start
                    wa_cost_basis = (np.array(investments) * np.array(cost_basis)).sum() / np.array(investments).sum()
                    taxable_basis.append(usd_balance * ((value / wa_cost_basis) - 1))
                    investments.clear()
                    cost_basis.clear()
                    # Tax-calc-end

                else:
                    btc_usd_balance = btc_balance * value
            else:
                btc_usd_balance = btc_balance * value

        daily_usd_position.append(usd_balance)
        daily_btc_position.append(btc_balance)
        daily_btc_usd_position.append(btc_usd_balance)
        daily_portfolio_position.append(usd_balance + btc_usd_balance)

        counter += 1

        ### print(counter)

    # Tax-pay-start
    if np.array(taxable_basis).sum() > 0:
        taxes += np.array(taxable_basis).sum() * tax_rate
        daily_portfolio_position[-1] -= taxes
    # Tax-pay-end

    ### print(np.array(taxable_basis).sum() * tax_rate)

    return pd.Series(daily_portfolio_position)

In [863]:
f"hodler: {list(play_hodler_strategy(y_true_bull, y_pred_bull).tail(1))[0] + list(play_hodler_strategy(y_true_bear, y_pred_bear).tail(1))[0] + list(play_hodler_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]}"

0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
3926.5519614387326
0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
2233.3142470034895
0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
3668.363693915314


'hodler: 9176.771161650995'

In [864]:
f"trader: {list(play_trader_strategy(y_true_bull, y_pred_bull).tail(1))[0] + list(play_trader_strategy(y_true_bear, y_pred_bear).tail(1))[0] + list(play_trader_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]}"

'trader: 10738.17905324752'

In [865]:
f"whale: {list(play_whale_strategy(y_true_bull, y_pred_bull).tail(1))[0] + list(play_whale_strategy(y_true_bear, y_pred_bear).tail(1))[0] + list(play_whale_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]}"

'whale: 10536.509087220698'

In [866]:
f"hodler_whale: {list(play_hodler_whale_strategy(y_true_bull, y_pred_bull).tail(1))[0] + list(play_hodler_whale_strategy(y_true_bear, y_pred_bear).tail(1))[0] + list(play_hodler_whale_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]}"

'hodler_whale: 10418.650398622696'

In [867]:
f"charles: {list(play_charles_strategy(y_true_bull, y_pred_bull).tail(1))[0] + list(play_charles_strategy(y_true_bear, y_pred_bear).tail(1))[0] + list(play_charles_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]}"

'charles: 10606.94125596468'

In [868]:
list(play_hodler_strategy(y_true_bull, y_pred_bull).tail(1))[0]
# buy only, every week

0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
3926.5519614387326


3536.1375543660834

In [869]:
list(play_hodler_strategy(y_true_bear, y_pred_bear).tail(1))[0]
# buy only, every week

0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
2233.3142470034895


2233.3142470034895

In [870]:
list(play_hodler_strategy(y_true_sideways, y_pred_sideways).tail(1))[0]
# buy only, every week

0
2766.6666666666665
1
2533.333333333333
2
2299.9999999999995
3
2066.666666666666
4
1833.3333333333328
5
1599.9999999999995
6
1366.6666666666663
7
1133.333333333333
8
899.9999999999998
9
666.6666666666665
10
433.3333333333332
11
199.9999999999999
12
3668.363693915314


3407.3193602814217

In [871]:
play_trader_strategy(y_true_bull, y_pred_bull).tail(1)
# buy 0.10, sell -0.05

89    3404.266547
dtype: float64

In [872]:
play_trader_strategy(y_true_bear, y_pred_bear).tail(1)
# buy 0.10, sell -0.05

89    3182.25653
dtype: float64

In [873]:
play_trader_strategy(y_true_sideways, y_pred_sideways).tail(1)
# buy 0.10, sell -0.05

89    4151.655976
dtype: float64

In [874]:
play_whale_strategy(y_true_bull, y_pred_bull).tail(1)
# buy 0.15, sell -0.05

89    3404.266547
dtype: float64

In [875]:
play_whale_strategy(y_true_bear, y_pred_bear).tail(1)
# buy 0.15, sell -0.05

89    3000.0
dtype: float64

In [876]:
play_whale_strategy(y_true_sideways, y_pred_sideways).tail(1)
# buy 0.15, sell -0.05

89    4132.24254
dtype: float64

In [877]:
play_hodler_whale_strategy(y_true_bull, y_pred_bull).tail(1)
# buy 0.15, sell -1

89    3943.49705
dtype: float64

In [878]:
play_hodler_whale_strategy(y_true_bear, y_pred_bear).tail(1)
# buy 0.15, sell -1

89    3000.0
dtype: float64

In [879]:
play_hodler_whale_strategy(y_true_sideways, y_pred_sideways).tail(1)
# buy 0.15, sell -1

89    3475.153349
dtype: float64

In [880]:
play_charles_strategy(y_true_bull, y_pred_bull).tail(1)
# buy 0.10, sell -0.10

89    3501.69503
dtype: float64

In [881]:
play_charles_strategy(y_true_bear, y_pred_bear).tail(1)
# buy 0.10, sell -0.10

89    3094.657854
dtype: float64

In [882]:
play_charles_strategy(y_true_sideways, y_pred_sideways).tail(1)
# buy 0.10, sell -0.10

89    4010.588372
dtype: float64